In [1]:
import netgen.gui
import traceback
import logging

%gui tk


# ngsolve stuff
from ngsolve import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

from netgen.csg import unit_cube

Our testing data

In [2]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.4, quad_dominated=False))
mesh3 = Mesh(unit_cube.GenerateMesh(maxh=1))
Draw(mesh)

The Faces are numbered differnttly than the Elements even though Elements are the Faces in 2D meshes by definition.

In [3]:
for e in mesh.Elements(VOL):
    print(e.nr, e.faces[0].nr)

0 0
1 4
2 6
3 7
4 8
5 9
6 10
7 11
8 12
9 13
10 14
11 15
12 1
13 18
14 19
15 20
16 21
17 22
18 23
19 24
20 25
21 26
22 27
23 28
24 2
25 31
26 32
27 33
28 34
29 35
30 36
31 37
32 38
33 39
34 40
35 41
36 3
37 44
38 45
39 46
40 47
41 48
42 49
43 50
44 51
45 52
46 5
47 16
48 29
49 42
50 53
51 54
52 58
53 56
54 59
55 62
56 64
57 65
58 17
59 70
60 68
61 71
62 30
63 72
64 73
65 74
66 75
67 80
68 78
69 81
70 43
71 82
72 83
73 84
74 85
75 55
76 90
77 88
78 91
79 92
80 93
81 76
82 60
83 61
84 63
85 66
86 86
87 57
88 87
89 69
90 67
91 79
92 77
93 94
94 95
95 89
96 100
97 103
98 104
99 107
100 109
101 110
102 113
103 115
104 97
105 96
106 105
107 108
108 116
109 111
110 114
111 117
112 102
113 101
114 118
115 106
116 112
117 98
118 99
119 120
120 119
121 123
122 122
123 121


Getting the associated elements from a face does not seem work, unlike one would expect, `len(f.elements)` might return 0 in 2D meshes for some faces.

however if it does not return 0, it always returns a one element list containing the same id as the id of the face.

In [4]:
#these two differnt ways to count the number of elements do not give the same result.
sum_elements = 0
sum_elements_2 = 0
sum_faces = 0
for f in mesh.Elements(VOL):
    sum_elements_2 += 1
for f in mesh.faces:
    if len(f.elements) > 0:
        print("f2e ", f, " ", f.elements[0])
    sum_elements += len(f.elements)
    sum_faces += 1
print("Num Elements:", sum_elements_2)
print("Num Faces:", sum_faces)
print("Num Elements associates to faces:", sum_elements)

f2e  F0   VEl 0
f2e  F86   VEl 86
f2e  F88   VEl 88
f2e  F94   VEl 94
f2e  F106   VEl 106
f2e  F107   VEl 107
f2e  F122   VEl 122
Num Elements: 124
Num Faces: 124
Num Elements associates to faces: 7


The other way however works. Getting the face id from an element.

In [5]:
elements_to_face = [e.faces[0].nr for e in mesh.Elements(VOL)]
for counter, f_id in enumerate(elements_to_face):
    if counter == f_id:
        print('element id matches face id at', counter)
#even for the faces where the face_to_element function form 
#the first block gave the same faceids and element ids the
#elements_to_face function returns differnt values
for index in [86, 88, 94, 106, 107, 122]:
    print('e2f', index, elements_to_face[index])


element id matches face id at 0
element id matches face id at 86
element id matches face id at 122
e2f 86 86
e2f 88 87
e2f 94 95
e2f 106 105
e2f 107 108
e2f 122 122


`mesh.Elements(VOL)` can ( unlike `mesh[NodeId(ELEMENT, i)]` as seen below) be used to give information about the verticies of an element, (`mesh[NodeId(FACE, i)]` can also do that).

In [6]:
try:
    a = next(x for x in mesh3.Elements(VOL))
    print(a.vertices)#this one works
except Exception as e:
    print(traceback.format_exc()) 

(V4, V5, V8, V7)


the types `NodeId(ELEMENT, 0)` and `NodeId(FACET, 0)` are basicially useless.

In [7]:
nodetyes = [VERTEX, EDGE, FACE, CELL, FACET, ELEMENT]
nodeattrs = ["vertices", "edges", "faces", "cells", "facets", "elements", "point"]
for t in nodetyes:
    desc =  str(t) + " has:"
    for a in nodeattrs:
        try:
            if hasattr(mesh[NodeId(t, 0)], a):
                desc = desc + " " + a
        except TypeError:
            pass
        except IndexError:
            #this test does not work for CELLS  if mesh is a 2d grid. 
            pass
    print(desc)
#for i in range(100):
#    print(len(mesh[NodeId(ELEMENT, i)].edges))
#help(mesh[NodeId(ELEMENT, 0)])

NODE_TYPE.VERTEX has: edges elements point
NODE_TYPE.EDGE has: vertices elements
NODE_TYPE.FACE has: vertices edges elements
NODE_TYPE.CELL has:
NODE_TYPE.FACET has:
NODE_TYPE.ELEMENT has:


as seen above, in the 2D case the numering of FACES and ELEMENTS are differnt 
and one can convert between ELEMENT numbers and FACE numbers by using 
`elements_to_face = [e.faces[0].nr for e in mesh.Elements(VOL)]`, since
this is clearly a 1:1 correspondance. However,
it is unclear whether the numering between CELLS and ELEMENTS is
also different in the 3D case and if yes, how to convert them since the
objects that Elements(VOL) iterates over does not seem to have a `cells` member

In [8]:
try:
    a = next(x for x in mesh3.Elements(VOL))
    print(a.cells)
except Exception as e:
    print(traceback.format_exc())

Traceback (most recent call last):
  File "<ipython-input-8-1ccc6313bd4a>", line 3, in <module>
    print(a.cells)
AttributeError: 'ngsolve.comp.Ngs_Element' object has no attribute 'cells'



`Draw()` fails silently when the name contains a space

In [9]:
def DrawTest():
    l2 = L2(mesh)
    gf_e = GridFunction(l2)
    gf_e.vec[0] = True
    Draw(gf_e, mesh, "first element")
DrawTest()